In [1]:
import fastai
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)

train_path = path/"train"
valid_path = path/'valid'

train_three = (train_path/'3').ls().sorted()
train_seven = (train_path/'7').ls().sorted()
valid_three = (valid_path/'3').ls().sorted()
valid_seven = (valid_path/'7').ls().sorted()

three_tens = [tensor(Image.open(o)) for o in train_three]
seven_tens = [tensor(Image.open(o)) for o in train_seven]
valid_three_tens = [tensor(Image.open(o)) for o in valid_three]
valid_seven_tens = [tensor(Image.open(o)) for o in valid_seven]

stacked_threes = torch.stack(three_tens).float()/255
stacked_sevens = torch.stack(seven_tens).float()/255
stacked_valid_threes = torch.stack(valid_three_tens).float()/255
stacked_valid_sev = torch.stack(valid_seven_tens).float()/255

In [6]:
mean_threes = stacked_threes.mean(0)
mean_sevens = stacked_sevens.mean(0)

In [7]:
def rms(true_mean, test_mean):
    return ((true_mean - test_mean)**2).mean().sqrt()

In [9]:
def is_3(m3, m7, img):
    return (rms(m3, img) < rms(m7, img)).item()
    

In [11]:
#is_3(mean_threes, mean_sevens, img7)

In [12]:
#img7 = stacked_sevens[7]
#plt.imshow(img7)

# sgd

In [4]:
def Linear1(xb):
    return xb@weights + bias

In [6]:
def init_params(param_size, std=1.):
    return (torch.randn(param_size, requires_grad=True))*std

weights = init_params((28*28,1))
bias = init_params(1)
weights.shape

torch.Size([784, 1])

torch.Size([784, 1])

In [16]:
params = weights, bias

In [3]:
x_train = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
y_train = tensor([1]*len(train_three) + [0]*len(train_seven)).unsqueeze(1).float()
x_train.shape, y_train.shape

valid_x = torch.cat([stacked_valid_threes, stacked_valid_sev]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_three) + [0]*len(valid_seven)).unsqueeze(1)
valid_x.shape, valid_y.shape

(torch.Size([2038, 784]), torch.Size([2038, 1]))

(torch.Size([2038, 784]), torch.Size([2038, 1]))

In [19]:
dl = DataLoader(x_train, batch_size=5, shuffle=True)

In [20]:
preds = Linear1(x_train)

In [21]:
preds.shape

torch.Size([12396, 1])

In [22]:
preds

tensor([[ -5.4640],
        [ -5.5140],
        [-12.8549],
        ...,
        [  1.8949],
        [ -8.1407],
        [  7.4801]], grad_fn=<AddBackward0>)

In [23]:
preds.sigmoid()

tensor([[4.2187e-03],
        [4.0139e-03],
        [2.6133e-06],
        ...,
        [8.6931e-01],
        [2.9136e-04],
        [9.9944e-01]], grad_fn=<SigmoidBackward>)

In [24]:
correct = (preds>0.0).float() == y_train
correct.count_nonzero(), y_train.count_nonzero()
correct.float().mean().item()

0.398031622171402

In [7]:
def mnist_loss(targets, prediction):
    prediction = prediction.sigmoid()
    return torch.where(targets==1, 1-prediction, prediction).mean()
    

In [26]:
z = mnist_loss(x_train[:5], preds[:5])

In [28]:
dset = zip(x_train, y_train)


In [29]:
dl = DataLoader(dset, batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [9]:
valid_dset = zip(valid_x, valid_y)

In [10]:
valid_dl = DataLoader(valid_dset, batch_size=250)
vb, vy = first(valid_dl)
vb.shape, vy.shape

(torch.Size([250, 784]), torch.Size([250, 1]))

In [13]:
batch = x_train[:4]
batch.shape

torch.Size([4, 784])

In [14]:
preds = Linear1(batch)
preds

tensor([[ -2.5633],
        [-11.0705],
        [ -0.7395],
        [-15.0158]], grad_fn=<AddBackward0>)

In [15]:
loss = mnist_loss(y_train[:4], preds)
loss

tensor(0.9013, grad_fn=<MeanBackward0>)

In [1]:
#loss.backward()
#weights.grad.shape, weights.grad.mean(), bias.grad 

In [43]:
def cal_grad(xb, yb, model):
    
    preds = model(xb)
    loss = mnist_loss(yb, preds)
    loss.backward()
    #for p,b in weights, bias:
        #p.data -= p.data * bias.data

In [45]:
cal_grad(batch, y_train[:4], Linear1)


In [38]:
cal_grad(batch, y_train[:4], Linear1)
weights.grad.mean(), bias.grad 

(tensor(-0.0010), tensor([-0.0066]))

In [39]:
def train_epoch(model, lr, params):
    for xb, yb, in dl:
        cal_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [40]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()

In [41]:
batch_accuracy(Linear1(batch), y_train[:4])

tensor(0.)

In [42]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [95]:
#validate_epoch(Linear1)

accs = [batch_accuracy(Linear1(xb), yb) for xb, yb in valid_dl]

In [96]:
l = [batch_accuracy(Linear1(batch), y_train[:5]) for xb, yb in valid_dl]

In [116]:
lr = 1.
params = weights, bias
train_epoch(Linear1, lr, params)
validate_epoch(Linear1)

0.7486

In [ ]:
for i in range(20):
    train_epoch(Linear1, lr, params)
    print(validate_epoch(Linear1), end=' ')

# creating an optimizer

In [ ]:
class linear:
    
    def __init__(self, out_shape, in_shape):
        self.out_shape = out_shape
        self.in_shape = in_shape
        
        
    def model(self):
        
        preds = x@weights + bias

In [84]:
linear_model = nn.Linear(28*28, 1)

In [85]:
w,b = linear_model.parameters()
w.shape, b.shape

(torch.Size([1, 784]), torch.Size([1]))

In [88]:
lr = 0.001

In [89]:
class BasicOptim:
    def __init__ (self, params, lr):
        self.params, self.lr = list(params), lr
        
    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr
            
    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None    

In [90]:
opt = BasicOptim(linear_model.parameters(), lr)

In [117]:
def train_epoch(model):
    
    for xb, yb in dl:
        cal_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

In [121]:
validate_epoch(linear_model)

0.2525

In [92]:
dls = DataLoaders(dl, valid_dl)

In [93]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [126]:
def simple_net(xb):
    res = xb@w1 + b1
    res = res.max(tensor(0.0))
    res = res@w2 + b2
    return res